In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import os
import warnings
import time
import EMGFeatureExtraction
warnings.filterwarnings('ignore')

In [2]:
random.seed(101)

# prepare data
datasets_path = 'data/gestures-9/'
datasets = []
for dataset_name in list(os.walk(datasets_path))[0][1]:
    dataset_path = datasets_path + dataset_name
    print(dataset_path)
    session_names = list(os.walk(dataset_path))[0][1]
    sessions = []
    for session_name in session_names:
        current_session = os.path.join(dataset_path, session_name)
        print(current_session)
        sessions.append(EMGFeatureExtraction.read_emg(current_session))
    datasets.append(sessions)
    print()

preprocessed_datasets = []
for dataset in datasets:
    preprocessed_sessions = []
    for session in dataset:
        window_samples = EMGFeatureExtraction.samples_num_in_window(200, 200)
        session_win = EMGFeatureExtraction.emg_data_windowing(session, window_samples)

        session_X = session_win[:,:-1]
        session_y = session_win[:,-1].astype('int')

        session_features = np.c_[session_X, session_y]
        preprocessed_sessions.append(session_features)
    preprocessed_datasets.append(preprocessed_sessions)

data/gestures-9/p001
data/gestures-9/p001\session1
input shape (2250, 65)
result shape:  (18000, 9)
data/gestures-9/p001\session2
input shape (2250, 65)
result shape:  (18000, 9)
data/gestures-9/p001\session3
input shape (2250, 65)
result shape:  (18000, 9)

data/gestures-9/p002
data/gestures-9/p002\session1
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p002\session2
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p002\session3
input shape (2251, 65)
result shape:  (18008, 9)

data/gestures-9/p003
data/gestures-9/p003\session1
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p003\session2
input shape (2251, 65)
result shape:  (18008, 9)
data/gestures-9/p003\session3
input shape (2251, 65)
result shape:  (18008, 9)



In [3]:
# time-domain features calculation time for one sample
session = preprocessed_datasets[0][0]
td_time = []
channels_num = 8
for sample in session:
    start_time = time.process_time()
    
    features = []
    for channel in range(8):
        channel_features = []
        channel_features.append(EMGFeatureExtraction.mean_absolute_value(sample[channel::channels_num]))
        channel_features.append(EMGFeatureExtraction.waveform_length(sample[channel::channels_num]))
        channel_features.append(EMGFeatureExtraction.zero_crossing(sample[channel::channels_num]))
        channel_features.append(EMGFeatureExtraction.slope_sign_changes(sample[channel::channels_num]))
        channel_features.append(EMGFeatureExtraction.root_mean_square(sample[channel::channels_num]))
        
        ar_order = 6
        ar_coef = np.array(EMGFeatureExtraction.autoregression_coefficients(sample[channel::channels_num], ar_order))
        channel_features += ar_coef.transpose().tolist()
        
        features += channel_features
    features = np.array(features).transpose()
                                
    td_time.append(time.process_time() - start_time)

print(f'mean time = {np.mean(td_time):.5f}')

mean time = 0.00201


In [4]:
# cwt calculation time for one sample
session = preprocessed_datasets[0][0]
cwt_time = []
channels_num = 8
for sample in session:
    start_time = time.process_time()
    
    features = []
    for channel in range(8):
        coef = EMGFeatureExtraction.calculate_CWT_vector(sample[channel::channels_num], 0.25)
        features.append(np.array(coef).transpose())
    features = np.array(features).transpose()
    
    cwt_time.append(time.process_time() - start_time)

print(f'mean time = {np.mean(cwt_time):.5f}')

mean time = 0.01479


In [5]:
# spectrogram calculation time for one sample
session = preprocessed_datasets[0][0]
spec_time = []
channels_num = 8
for sample in session:
    start_time = time.process_time()
    
    features = []
    for channel in range(8):
        coef = EMGFeatureExtraction.spectrogram_vector(sample[channel::channels_num], 200, npserseg=21, noverlap=18)
        features.append(np.array(coef).transpose())
    features = np.array(features).transpose()
    
    spec_time.append(time.process_time() - start_time)

print(f'mean time = {np.mean(spec_time):.5f}')

mean time = 0.00125
